## Pozyskanie danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne pakiety

In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery

## <center>Część 2</center>
### 2.5

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'projekt1-415610-f19ebf54e064.json'  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

### Potrzebna funkcja

In [ ]:
def return_df(q):
    query_job = client.query(q)
    query_result = query_job.result()
    return query_result.to_dataframe()

### 2.6

In [ ]:
query = 'select * from `bigquery-public-data.covid19_open_data.covid19_open_data` limit 10'
df = return_df(query)

### Informacje o zbiorze

In [ ]:
df.info()

## <center>Część 3</center>

### 3.1. Sprawdź, ile jest zapisanych wierszy z danymi.

In [ ]:
df_3_1 = return_df('SELECT COUNT(*) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_1

### 3.2. Sprawdź, ile krajów jest uwzględnionych w danych.

In [ ]:
df_3_2 = return_df('SELECT COUNT(DISTINCT country_name) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_2

### 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.

In [ ]:
df_3_3 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\' AND column_name LIKE \'%new_%\'')
df_3_3

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [ ]:
df_3_4 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\' AND data_type = \'INT64\' OR data_type = \'FLOAT64\''
)
df_3_4

### 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [ ]:
queries_3_5 = [
    'SELECT MIN(date) AS min_date, MAX(date) AS max_date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`',
    'SELECT MIN(date) AS min_date_confirmed, MAX(date) AS max_date_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_confirmed IS NOT NULL',
    'SELECT MIN(date) AS min_date_deceased, MAX(date) AS max_date_deceased FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_deceased IS NOT NULL',
    'SELECT MIN(date) AS min_date_vaccinated, MAX(date) AS max_date_vaccinated FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_persons_vaccinated IS NOT NULL'
]

df_3_5 = pd.DataFrame()

for query in queries_3_5:
    df_3_5 = pd.concat([df_3_5, return_df(query)], axis=1)
df_3_5

### 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń.

In [ ]:
queries_3_6 = [
    'SELECT country_name, infant_mortality_rate, adult_male_mortality_rate, adult_female_mortality_rate, ROUND(nurses_per_1000 + physicians_per_1000, 2) AS medical_personnel FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND adult_male_mortality_rate IS NOT NULL AND adult_female_mortality_rate IS NOT NULL GROUP BY country_name, infant_mortality_rate, adult_male_mortality_rate, adult_female_mortality_rate, gdp_usd, nurses_per_1000, physicians_per_1000 ORDER BY gdp_usd DESC LIMIT 10',
    'SELECT subregion1_name, date, testing_policy FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 1 AND country_name=\'Canada\' GROUP BY subregion1_name, date, testing_policy ORDER BY subregion1_name, date',
    'SELECT subregion2_name, date, CAST(new_tested AS int64) AS new_tested, new_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 2 AND subregion1_name=\'Łódź\' GROUP BY subregion2_name, date, new_tested, new_confirmed ORDER BY subregion2_name, date',
    'SELECT date, facial_coverings FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_name=\'Germany\' AND aggregation_level = 0 GROUP BY date, facial_coverings ORDER BY date',
    'SELECT date, workplace_closing, income_support, CAST(mobility_workplaces AS int64) AS mobility_workplaces FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND country_name=\'United States of America\' GROUP BY date, workplace_closing, income_support, mobility_workplaces ORDER BY date'
]

for query in queries_3_6:
    df_3_6 = return_df(query)
    display(df_3_6.head())

## <center>Część 4</center>

### 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [ ]:
df_4 = [return_df('SELECT iso_3166_1_alpha_3 AS CCA3, country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, health_expenditure_usd, life_expectancy, population_age_00_09 AS age_bin_0, population_age_10_19 AS age_bin_1, population_age_20_29 AS age_bin_2, population_age_30_39 AS age_bin_3, population_age_40_49 AS age_bin_4, population_age_50_59 AS age_bin_5, population_age_60_69 AS age_bin_6, population_age_70_79 AS age_bin_7, population_age_80_and_older AS age_bin_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY iso_3166_1_alpha_3, country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, health_expenditure_usd, life_expectancy, population_age_00_09, population_age_10_19, population_age_20_29, population_age_30_39, population_age_40_49, population_age_50_59, population_age_60_69, population_age_70_79, population_age_80_and_older ORDER BY country_name')]

df_4[0].head()

### Uzupełnienie brakujących danych o kraju na podstawie jego <i>sąsiadów</i>

In [ ]:
from haversine import haversine

def save_to_csv(dataframe, filename):
    if 'czesc4' not in os.listdir(os.getcwd()):
        os.mkdir('czesc4')
    os.chdir('czesc4')
    dataframe.to_csv(filename + '.csv', index=False) # Zapis do csv
    os.chdir('..')


def fill_based_on_neigh(index, columns, num=10):
    data = df_4[0].loc[index].copy()
    df_4[0]['distance'] = df_4[0].apply(lambda x: haversine((data['latitude'], data['longitude']), (x['latitude'], x['longitude'])), axis=1)
    neigh = df_4[0].sort_values(by=['distance'])
    neigh = neigh.iloc[0:num]
    for col in columns:
        if df_4[0][col].dtype == np.float64:
            df_4[0].loc[index, col] = neigh[col].mean()
        else:
            df_4[0].loc[index, col] = int(neigh[col].mean())


def fill_4_1():
    columns = df_4[0].columns
    age_groups = [el for el in columns if 'age_bin_' in el]
    leave = ['CCA3', 'population', 'population_male', 'population_female', 'latitude', 'longitude', 'country_name'] + age_groups
    
    df_4[0].loc[df_4[0]['country_name'] == 'Armenia', ['latitude', 'longitude']] = (40.17720, 44.50349)
    
    df_4[0]['distance'] = 0.0
    to_float_cols = columns.difference(leave)
    for tfc in to_float_cols:
         df_4[0][tfc] = df_4[0][tfc].astype(np.float64)
    
    df_4[0]['population'].fillna(df_4[0]['population_male'] + df_4[0]['population_female'], inplace=True)
    df_4[0]['population_male'].fillna(df_4[0]['population'] // 2, inplace = True)
    df_4[0]['population_female'].fillna(df_4[0]['population'] // 2, inplace = True)
    
    not_equal = df_4[0]['population'] != df_4[0]['population_male'] + df_4[0]['population_female']
    df_4[0]['population'].mask(not_equal, df_4[0]['population_male'] + df_4[0]['population_female'], inplace=True)
    
    for index, row in df_4[0].iterrows():
        if df_4[0].loc[index].isna().sum() > 0:
            fill_based_on_neigh(index=index, columns=df_4[0].columns.difference(leave))
    
    df_4[0].drop('distance', axis=1, inplace=True)
    split = [9, 9, 8, 7, 7, 8, 9, 9]
    for i in range(len(split)):
        df_4[0][age_groups[i]].fillna(df_4[0]['population'] // split[i], inplace=True)
    df_4[0]['age_bin_8'].fillna(df_4[0]['population'] - df_4[0][age_groups].sum(axis=1), inplace=True)


fill_4_1()
save_to_csv(df_4[0], '4_1')

df_4[0].info()

### Informacje o krajach

In [ ]:
df_4[0].head()

### 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [ ]:
df_4.append(return_df('SELECT country_name, date, new_confirmed AS daily_confirmed, new_confirmed_male AS daily_confirmed_male, new_confirmed_female AS daily_confirmed_female, new_confirmed_age_0 AS age_confirmed_0, new_confirmed_age_1 AS age_confirmed_1, new_confirmed_age_2 AS age_confirmed_2, new_confirmed_age_3 AS age_confirmed_3, new_confirmed_age_4 AS age_confirmed_4, new_confirmed_age_5 AS age_confirmed_5, new_confirmed_age_6 AS age_confirmed_6, new_confirmed_age_7 AS age_confirmed_7, new_confirmed_age_8 + new_confirmed_age_9 AS age_confirmed_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL ORDER BY country_name, date'))

df_4[1].info()

### Uzupełnienie danych dotyczących zachorowań na COVID-19

In [ ]:
def fill_values(dataframe):
    columns = dataframe.columns
    
    dataframe[columns[2]].fillna(0, inplace=True)
    
    male_and_female_nan = dataframe[columns[3]].isna() & dataframe[columns[4]].isna()
    rand = np.random.randint(1, 10, size=len(male_and_female_nan))
    dataframe[columns[3]].mask(male_and_female_nan, dataframe[columns[2]] // rand, inplace=True)
    dataframe[columns[4]].mask(male_and_female_nan, dataframe[columns[2]] - dataframe[columns[3]], inplace=True)
    
    only_male_nan = dataframe[columns[3]].isna() & dataframe[columns[4]].notna()
    dataframe[columns[3]].mask(only_male_nan, dataframe[columns[2]] - dataframe[columns[4]], inplace=True)
    
    only_female_nan = dataframe[columns[4]].isna() & dataframe[columns[3]].notna()
    dataframe[columns[4]].mask(only_female_nan, dataframe[columns[2]] - dataframe[columns[3]], inplace=True)
    
    all_is_zero = (dataframe[columns[2]] == 0) & (dataframe[columns[3]] != 0) & (dataframe[columns[4]] != 0)
    dataframe[columns[2]].mask(all_is_zero, dataframe[columns[3]] + dataframe[columns[4]], inplace=True)
    
    # dataframe = most_complete(dataframe)  # można użyć, ale lepiej nie
    
    ratio = pd.DataFrame({'country_name': df_4[0]['country_name']})
    
    age_groups_df = [col for col in columns if 'age_' in col]
    age_groups = [col for col in df_4[0].columns if 'age_' in col]
    
    for i in range(len(age_groups_df)):
        ratio[age_groups_df[i]] = df_4[0][age_groups[i]] / df_4[0]['population']
    dataframe = dataframe.merge(ratio, on='country_name', how='left')
    for i in range(len(age_groups_df)):
        dataframe[age_groups_df[i] + '_x'].fillna((dataframe[columns[2]] * dataframe[age_groups_df[i] + '_y']).astype(np.int64), inplace=True)
    dataframe.drop(columns=[age_group + '_y' for age_group in age_groups_df], axis=1, inplace=True)
    dataframe.rename(columns={age_group + '_x': age_group for age_group in age_groups_df}, inplace=True)
    
    return dataframe


def most_complete(dataframe):  # nie interesują nas kraje mających tylko puste rekordy w całym okresie zbierania danych
    nans = dataframe[dataframe.isna().any(axis=1)]['country_name'].value_counts()
    nans = nans[nans.values == 991].index
    return dataframe[~dataframe['country_name'].isin(nans)].reset_index(drop=True)

df_4[1] = fill_values(df_4[1])
save_to_csv(df_4[1], '4_2')

df_4[1].info()

### Miesięczne statystyki zachorowań na COVID-19 dla każdego kraju

In [ ]:
df_4[1].head()

### 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [ ]:
df_4.append(return_df('SELECT country_name, date, new_deceased AS daily_deceased, new_deceased_male AS daily_deceased_male, new_deceased_female AS daily_deceased_female, new_deceased_age_0 AS age_deceased_0, new_deceased_age_1 AS age_deceased_1, new_deceased_age_2 AS age_deceased_2, new_deceased_age_3 AS age_deceased_3, new_deceased_age_4 AS age_deceased_4, new_deceased_age_5 AS age_deceased_5, new_deceased_age_6 AS age_deceased_6, new_deceased_age_7 AS age_deceased_7, new_deceased_age_8 + new_deceased_age_9 AS age_deceased_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL ORDER BY country_name, date'))

df_4[2].info()

### Uzupełnienie danych dotyczących śmiertelności COVID-19

In [ ]:
df_4[2] = fill_values(df_4[2])
save_to_csv(df_4[2], '4_3')

df_4[2].info()

### Miesięczne statystyki śmierci na COVID-19

In [ ]:
df_4[2].head()

### 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

In [ ]:
df_4.append(return_df('SELECT country_name, date, new_persons_vaccinated AS daily_persons_vaccinated, new_persons_fully_vaccinated AS daily_persons_fully_vaccinated FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL ORDER BY country_name, date'))

df_4[3].info()

### Uzupełnienie danych dotyczących szczepień

In [ ]:
def fill_4_4(dataframe):
    dataframe['daily_persons_vaccinated'].fillna(0, inplace=True)
    dataframe['daily_persons_fully_vaccinated'].fillna(0, inplace=True)
    return dataframe


df_4[3] = fill_4_4(df_4[3])
save_to_csv(df_4[3], '4_4')

df_4[3].info()

### Miesięczne statystyki szczepień na COVID-19

In [ ]:
df_4[3].head()

### 4.5. Chcemy przeprowadzić skuteczność polityk zamykania szkół, a zachorowaniami w grupach wiekowych 0-9 i 10-19

In [ ]:
df_4.append(return_df('SELECT country_name, date, school_closing, new_confirmed_age_0 AS preschoolers_confirmed, new_confirmed_age_1 AS pupils_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL ORDER BY country_name, date, school_closing'))

df_4[4].info()

### Czyszczenie danych

In [ ]:
def fill_4_5(dataframe):
    dataframe['school_closing'].ffill(inplace=True)
    dataframe['school_closing'].fillna(-1, inplace=True)  # kiedy jeszcze nie było żadnych ograniczeń
    dataframe['preschoolers_confirmed'].fillna(0, inplace=True)
    dataframe['pupils_confirmed'].fillna(0, inplace=True)
    return dataframe


df_4[4] = fill_4_5(df_4[4])
save_to_csv(df_4[4], '4_5')

df_4[4].info()

### Miesięczne statystyki zachorowań w populacji do 19 roku życia

In [ ]:
df_4[4].head()

## <center>Część 5</center>
### Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

### Połączenie części 4

In [ ]:
df_5 = pd.merge(df_4[1], df_4[2], on=['country_name', 'date'])
df_5 = pd.merge(df_5, df_4[3], on=['country_name', 'date'])
df_5 = pd.merge(df_5, pd.merge(df_4[0], df_4[4], on=['country_name']), on=['country_name', 'date'])

df_5.info()

In [ ]:
df_5.head()

## <center>Część 6</center>

### 6.1

In [ ]:
base_1 = pd.read_csv('czesc6/world_countries.csv')
df_6 = pd.merge(base_1, df_5, on=['CCA3'])

df_6.info()

In [ ]:
df_6.head()

### 6.2

In [ ]:
base_2 = pd.read_csv('czesc6/gdp.csv')
base_2.rename(columns={'Country Name': 'country_name', 'Country Code': 'CCA3'}, inplace=True)
df_6 = pd.merge(base_2, df_6, on=['country_name', 'CCA3'])
save_to_csv(df_6, '6')

df_6.info()

In [ ]:
df_6.head()

## <center>Część 7</center>
### Znajdź 3 dodatkowe zbiory danych, które uważasz, że rozwijają w ciekawy sposób temat ostatniej pandemii. Powtórz dla nich analogiczne czynności z części 4, a następnie połącz je z danymi z części 6. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

### 1. Kurs walut europejskich wobec dolara w latach 2020-2022

In [ ]:
import requests  # biblioteka od realizacji żądań HTTP
from bs4 import BeautifulSoup  # biblioteka od web-scrapingu

def web_scrape(url, table_class):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    indiatable = soup.find('table', {'class': table_class})
    dataframe = pd.read_html(str(indiatable))
    return pd.DataFrame(dataframe[0])

### Przygotowanie danych o walutach światowych

In [ ]:
def prepare_currencies(dataframe):
    dataframe.dropna(inplace=True)
    dataframe['Country'] = dataframe['Country'].str.title()  # tylko pierwsza litera z dużej
    dataframe['Country'] = dataframe['Country'].apply(lambda x: x.split(' (')[0])  # usuń zbędne znaki
    dataframe.drop(['Number'], axis=1, inplace=True)  # usunięcie zbędnej kolumny
    dataframe['Country'] = dataframe['Country'].str.split(', ')  # rozbicie wierszy na poszczególne kraje
    dataframe = dataframe.explode('Country')
    dataframe.loc[dataframe['Country'] == 'Russian Federation', 'Country'] = 'Russia'
    dataframe.loc[dataframe['Country'] == 'Republic Of North Macedonia', 'Country'] = 'Macedonia'
    dataframe.loc[dataframe['Country'] ==
                  'United Kingdom Of Great Britain And Northern Ireland', 'Country'] = 'United Kingdom'

    # dataframe.to_csv('czesc7/world_curriencies.csv', index=False)
    return dataframe

world_currencies = web_scrape('https://www.iban.com/currency-codes', 'table table-bordered downloads tablesorter')
europe_countries = ['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia And Herzegovina', 'Bulgaria',
                    'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
                    'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein', 'Lithuania',
                    'Luxembourg', 'Macedonia', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
                    'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino',
                    'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom']

world_currencies = prepare_currencies(world_currencies)
world_currencies = world_currencies[world_currencies['Country'].isin(europe_countries)].reset_index(drop=True)
world_currencies.drop(index=[41, 43], inplace=True)  # 2 waluty z Szwajcarii nie rozpoznawane są
world_currencies.reset_index(drop=True, inplace=True)

world_currencies.info()

In [ ]:
world_currencies.head()

### Wypełnienie arkuszy Google Sheets w kursy walut

In [ ]:
import gspread

def update_sheets():
    for i, el in enumerate(curr):
        worksheet = sh.add_worksheet(title=f'{el}', rows=1097, cols=26, index=i)
        worksheet.update_cell(1, 1, f'=GOOGLEFINANCE("{el}"; "price"; DATE(2020;1;1); DATE(2023;1;1))')

def get_sheets_data():
    sheets = [sh.worksheet(el) for el in curr]
    result = pd.DataFrame()
    for sheet in sheets:
        data = sheet.get_values()[1:]
        sheet_data = pd.DataFrame(data, columns=['date', sheet.title])
        sheet_data['date'] = pd.to_datetime(sheet_data['date']).dt.date
        result = sheet_data if result.empty else result.merge(sheet_data, on='date', how='outer')
    # result.to_csv('czesc7/currency_data.csv', index=False)
    return result

def del_worksheets():
    sheets = [sh.worksheet(el) for el in curr]
    for sheet in sheets:
        sh.del_worksheet(sheet)

gc = gspread.service_account()
sh = gc.open('Currencies')

curr = world_currencies['Code'].unique().tolist()
curr = ['USD' + el for el in curr]
update_sheets()
currencies_data = get_sheets_data()
del_worksheets()

currencies_data.info()

In [ ]:
currencies_data.head()

### Transformacja tych danych

In [ ]:
currencies_data = currencies_data.ffill(axis=1)
currencies_data['date'] = pd.to_datetime(currencies_data['date'])
convert = [col for col in currencies_data.columns if 'USD' in col]
for col in convert:
    currencies_data[col] = currencies_data[col].str.replace(',', '.')
    currencies_data[col] = currencies_data[col].astype(np.float64)

currencies_data.info()

In [ ]:
currencies_data.head()

### 2. Tygodniowa cena benzyny w USA i UK w latach 2020-2022

In [ ]:
gasoline_US = pd.read_csv('czesc7/Weekly_Gasoline_Prices_USA.csv')
gasoline_UK = pd.read_excel('czesc7/Weekly_Gasoline_Prices_UK.xlsx')

gasoline_US.info()
gasoline_UK.info()

In [ ]:
display(gasoline_US.head())
display(gasoline_UK.head())

### Przygotowanie danych o cenach benzyny

In [ ]:
def restrict_to_min_max_date(dataframe, cca3, rename_columns):
    dataframe.rename(columns=rename_columns, inplace=True)
    if dataframe['date'].dtype != 'datetime64[ns]':
        dataframe['date'] = pd.to_datetime(dataframe['date'])
    min_dt, max_dt = pd.to_datetime(df_3_5['min_date'].values[0]), pd.to_datetime(df_3_5['max_date'].values[0])
    dataframe = dataframe[dataframe['date'].between(min_dt, max_dt)].reset_index(drop=True)
    dataframe['CCA3'] = cca3
    return dataframe

gasoline_US = restrict_to_min_max_date(gasoline_US, 'USA', {'Week of': 'date', 'U.S. Retail Gasoline Prices $/gal': 'Gasoline Prices $/litre'})
gasoline_US['Gasoline Prices $/litre'] /= 3.78541178

gasoline_UK = restrict_to_min_max_date(gasoline_UK, 'GBR', {'Date': 'date', 'ULSP:  Pump price (p/litre)': 'Gasoline Prices $/litre'})
gasoline_UK = pd.merge(gasoline_UK, currencies_data[['date', 'USDGBP']], on='date', how='inner')
gasoline_UK['Gasoline Prices $/litre'] = gasoline_UK['Gasoline Prices $/litre'] / (gasoline_UK['USDGBP'] * 100)
gasoline_UK['Gasoline Prices $/litre'] = gasoline_UK['Gasoline Prices $/litre'].apply(lambda x: round(x, 2))
gasoline_UK.drop(columns=['USDGBP'], axis=1, inplace=True)

gasoline = pd.concat([gasoline_US, gasoline_UK], axis=0)

gasoline.info()

In [ ]:
gasoline.head()

### 3. Przestępstwa w Południowej Australii w trakcie pandemii

In [ ]:
crimes_south_aus = pd.DataFrame()
for fname in os.listdir('czesc7/crime'):
    crimes_south_aus = pd.concat([crimes_south_aus, pd.read_csv('czesc7/crime/' + fname)], axis=0)

crimes_south_aus.info()

In [ ]:
crimes_south_aus.head()

### Przygotowanie danych o przestępstwach

In [ ]:
crimes_south_aus.drop(columns=['Offence Level 1 Description', 'Postcode - Incident'], axis=1, inplace=True)
crimes_south_aus.rename(columns={'Reported Date': 'date', 'Offence Level 2 Description': 'Offence Type', 'Offence Level 3 Description': 'Offence Description', 'Suburb - Incident': 'Suburb'}, inplace=True)
crimes_south_aus['date'] = pd.to_datetime(crimes_south_aus['date'], dayfirst=True)
crimes_south_aus.dropna(inplace=True)
crimes_south_aus['Offence count'] = crimes_south_aus['Offence count'].astype(np.int64)
crimes_south_aus['CCA3'] = 'AUS'

crimes_south_aus.info()

In [ ]:
crimes_south_aus.head()

### 4. Ceny energii w Belgii 2018-2024

In [ ]:
energy_stock_prices = pd.read_csv('czesc7/Energy_Stock_Prices_Belgium.csv', delimiter=';')

energy_stock_prices.info()

In [ ]:
energy_stock_prices.head()

### Przygotowanie danych o giełdowych cenach energii

In [ ]:
energy_stock_prices['Datetime'] = pd.to_datetime(energy_stock_prices['Datetime'], utc=True).dt.date
energy_stock_prices.rename(columns={'Datetime': 'date', 'Implicit net position': 'Stock Price'}, inplace=True)
min_date = df_3_5['min_date'].values[0]
max_date = df_3_5['max_date'].values[0]
energy_stock_prices = energy_stock_prices[energy_stock_prices['date'].between(min_date, max_date)].reset_index(drop=True)
energy_stock_prices['Stock Price'] = energy_stock_prices.groupby('date')['Stock Price'].transform('sum')
energy_stock_prices = energy_stock_prices.groupby('date').agg({'Stock Price': 'mean'}).reset_index()
energy_stock_prices['CCA3'] = 'BEL'

energy_stock_prices.info()

In [ ]:
energy_stock_prices.head()

### Połączenie z częścią 6

In [ ]:
def do_merge(dataframe, on1=None, on2=None, name=None):
    df_7 = pd.merge(dataframe, base_1, on=[on1])
    df_7 = pd.merge(df_7, base_2, on=[on2])
    save_to_csv(df_7, name)
    return df_7

gas = do_merge(gasoline, on1='CCA3', on2='CCA3', name='gasoline_US_UK')
crimes = do_merge(crimes_south_aus, on1='CCA3', on2='CCA3', name='crimes_south_aus')
energy = do_merge(energy_stock_prices, on1='CCA3', on2='CCA3', name='energy_stock_prices')

### Ostateczny wynik

In [ ]:
gas.head()

In [ ]:
crimes.head()

In [ ]:
energy.head()